In [ ]:
using PowerModels
using Ipopt
using ForwardDiff
using OrdinaryDiffEq
using NLsolve

display(HTML("<style>.container { width:100% !important; }</style>"))

data = PowerModels.parse_file("C:/Users/Ewa/Downloads/Twogen2bus.m")
pf_result = run_ac_pf(data ,Ipopt.Optimizer)
#PowerModels.print_summary(pf_result["solution"])

# Load flow results

# bus results
va1 =pf_result["solution"]["bus"]["1"]["va"]
va2 =pf_result["solution"]["bus"]["2"]["va"]
vm1 =pf_result["solution"]["bus"]["1"]["vm"]
vm2 =pf_result["solution"]["bus"]["2"]["vm"];

# Voltage at bus 1
V_tR = vm1*cos(va1)
V_tI = vm1*sin(va1)


# Voltage at bus 2
V_tR1 = vm2*cos(va2)
V_tI1 = vm2*sin(va2)

# generator results
pg = pf_result["solution"]["gen"]["1"]["pg"]
qg =pf_result["solution"]["gen"]["1"]["qg"]

# branch results
p_ref =pf_result["solution"]["branch"]["1"]["pf"]
q_ref =pf_result["solution"]["branch"]["1"]["qf"]
p_ref1 =pf_result["solution"]["branch"]["1"]["pt"]
q_ref1 =pf_result["solution"]["branch"]["1"]["qt"];

In [ ]:
# Define the parameters
lf =0.08
rf = 0.003
cf = 0.074
rt = 0.2
lt = 0.01

rl = 0.01

Rp = 0.1
Rq =0.1
ωz = 2*pi*5.0
ωz1 = 2*pi*5.0
ωg = 1.0

kvp = 0.52
kvi = 1.16
kvf = 1.0
rv = 0.0
lv =0.2
kip = 0.74
kii = 1.19
kif = 1.0

ω_ref = 1.0
v_ref = 0.8
ωn = 2*pi*50.0
ωb1 = 2*pi*50.0
ω_star = ωg


In [ ]:
function system!(F,x)
   
    
### Grid forming equations
  #𝜕eg/𝜕t
F[1]= (ωb1 / cf * x[3] - ωb1 / cf * x[5] + ωb1 * ωg * x[2])
   #𝜕eq/𝜕t
F[2]= (ωb1 / cf * x[4] - ωb1 / cf * x[6] - ωb1 * ωg *x[1])
   #𝜕isd/𝜕t
F[3]=(ωb1 / lf * (  kip * ( (kvp * (((v_ref + Rq*(q_ref - x[8]))- rv * x[5] + (ω_star + Rp*(p_ref - x[7])) * lv * x[6]) -x[1]) + kvi * x[10] - cf *  (ω_star + Rp*(p_ref - x[7])) * x[2] + kif * x[5]) - x[3]) + kii * x[12] - (ω_star + Rp*(p_ref - x[7])) * lf * x[4] + kvf *x[1])  - ωb1 / lf *x[1] - ωb1 * rf / lf * x[3] + ωb1 * ωg * x[4])
   #𝜕isq/𝜕t
F[4]=    (ωb1 / lf * ((kip * ((kvp * ((-rv * x[6] - (ω_star + Rp*(p_ref - x[7])) * lv * x[5]) - x[2]) + kvi * x[11] + cf *  (ω_star + Rp*(p_ref - x[7])) *x[1] + kif * x[6]) - x[4]) + kii * x[13] +  (ω_star + Rp*(p_ref - x[7]))* lf * x[3] + kvf * x[2] )) - ωb1 / lf * x[2] - ωb1 * rf / lf * x[4] - ωb1 * ωg * x[3])
   #𝜕igd/𝜕t
F[5]=  (ωb1 / lt *x[1] - ωb1 / lt * (rl*x[6]) - ωb1 * rt / lt * x[5] + ωb1 * ωg * x[6])
   #𝜕igq/𝜕t
F[6]= (ωb1 / lt * x[2] + ωb1 / lt * (rl*x[5]) - ωb1 * rt / lt * x[6] - ωb1 * ωg * x[5])
   # Outer control model - virtual inertia
   #𝜕pm/𝜕t
F[7]=  (ωz1*x[6]*x[2] + ωz1*x[5]*x[1]- ωz1*x[7])
   #𝜕qm/𝜕t
F[8]= (-ωz1 * x[6] *x[1] + ωz1 * x[5] * x[2] - ωz1 * x[8])
  #𝜕δc/𝜕t
F[9]= ωb1*(ω_star + Rp*(p_ref - x[7]))
   # Voltage control
   #PI Integrator (internal state)
   #𝜕ξ_d/𝜕t
F[10]=     (((v_ref + Rq*(q_ref - x[8]))  - rv * x[5] +  (ω_star + Rp*(p_ref - x[7])) * lv * x[6]) -x[1])
   #𝜕ξ_q/𝜕t
F[11]=     ((-rv * x[6] - (ω_star + Rp*(p_ref - x[7])) * lv * x[5]) - x[2])
   # Current control
   # PI integrator (internal state)
   #𝜕γ_d/𝜕t
F[12]=(kvp * (((v_ref + Rq*(q_ref - x[8])) - rv * x[5] +  (ω_star + Rp*(p_ref - x[7])) * lv * x[6]) -x[1]) + kvi * x[10] - cf *  (ω_star + Rp*(p_ref - x[7])) * x[2] + kif * x[5]) - x[3]
    #𝜕γ_q/𝜕t
F[13]= (kvp * ((-rv * x[6] -  (ω_star + Rp*(p_ref - x[7])) * lv * x[5]) - x[2]) + kvi * x[11] + cf * (ω_star + Rp*(p_ref - x[7])) *x[1] + kif * x[6]) - x[4]
end


# Initial Conditions
     initial =  [
           V_tR1, #egd1
           V_tI1, #egq1
            p_ref/V_tR1, #isd1
            0.0, #isq1
            p_ref/V_tR1, #igd1
            0.0,#igq1
            p_ref, #pm1
            q_ref, #qm1
            0.0,#δc1
            0.0, #ξ_d1
            0.0, #ξ_q1
            0.0, #γ_d1
            0.0, #γ_q1
         ]


         sys_solve = NLsolve.nlsolve(
                     system!,
                     initial,
                    xtol = :1e-9,
                     ftol = :1e-9,
                     method = :trust_region,
                    ) #Solve using initial guess x0
                      print(sys_solve)